### Installing the required libraries

In [1]:
#pip install beautifulsoup4
#pip install lxml
#pip install requests

### Importing the required libraries
re is used to clean the str data, with the .split() method it was found challenging and time consuming to remove '\r' and '\n' strings from scraped data (when appending to the .csv file).

In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import re
import numpy as np
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

### Setting the required variables
Also, creating the soup object from which the table data is to be parsed

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
article = soup.find('body')
table = article.find('table', class_='wikitable sortable').tbody

### Creating a .csv file, iterating through the soup table object and parsing data, appending the data, and closing the .csv file

In [4]:
#Creates .csv file
csv_file = open('toronto_data.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode','Borough', 'Neighborhood'])

#Iterates through table to extract, parse, and append data
for row in table.find_all('tr'):
    table_row = row

    list = []
    
    for cell in row.find_all('td'):
        table_cell = re.sub(r'(\r+|\n)', '', cell.text)
        list.append(table_cell)
    
    csv_writer.writerow(list)

#Closes the csv file
csv_file.close()

### Taking a look at the raw data prior to processing / cleaning

In [5]:
raw_dataset = pd.read_csv('toronto_data.csv')
raw_dataset.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Filtering the out raw data where Borough is not assigned as instructed

In [6]:
# Get names of indexes for which Borough = 'Not assigned'
indexNames = raw_dataset[raw_dataset['Borough'] == 'Not assigned' ].index

# Delete these row indexes from dataFrame
raw_dataset.drop(indexNames , inplace=True)

#Quick inspection
raw_dataset.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Looks good!

### And, setting not assigned neighborhoods to the borough value
...upon inspection it was found that no neighborhoods were set to "not assigned", a few had NaN values, however, all of these were attributed to observations where the Borough was not assigned. As such, they were filtered out, and this task is done.

### Reseting index and printing shape of df

In [7]:
raw_dataset.reset_index(drop=True, inplace=True)
print(f'The raw data set is of dimensions {raw_dataset.shape}')

The raw data set is of dimensions (103, 3)


In [8]:
# Check that index is good
raw_dataset.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Importing the geocoder

In [9]:
#pip install geocoding
import geocoder

In [10]:
'''# initialize your variable to None
lat_lng_coords = None

postal_code = 'MG5'

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]'''

"# initialize your variable to None\nlat_lng_coords = None\n\npostal_code = 'MG5'\n\n# loop until you get the coordinates\nwhile(lat_lng_coords is None):\n  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n  lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]"

### I was finding that the while loop above was running infinitely.
Because of this, I instead used the provided csv file with the data provided.

In [11]:
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Setting up the final df

In [12]:
#Creating two more columns for Latitude and Longitude, and initializing values as None (for post-processing inspection purposes)
raw_dataset['Latitude'] = None
raw_dataset['Longitude'] = None

raw_dataset.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",None,None
3,M6A,North York,"Lawrence Manor, Lawrence Heights",None,None
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",None,None


### Using enumerate to return a tuple of both index and str value, the code below iterates through and sets the appropriate values to the latitude and longitude!

In [13]:
for postCode in enumerate(raw_dataset['PostalCode']):
    
    #retrieving the index of the postal code
    geospatial_index = geospatial_data.index[geospatial_data['Postal Code'] == postCode[1]].tolist()[0]
    
    #setting the latitude
    raw_dataset.loc[postCode[0], 'Latitude'] = geospatial_data.iloc[geospatial_index][1]
    
    #setting the longtitude
    raw_dataset.loc[postCode[0], 'Longitude'] = geospatial_data.iloc[geospatial_index][2]

### Quick inspection below (looks good!)

In [14]:
raw_dataset.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895


### Now to visualise it all....

In [15]:
from geopy.geocoders import Nominatim
import folium

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(raw_dataset['Latitude'], raw_dataset['Longitude'], raw_dataset['Borough'], raw_dataset['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Exploring with FourSquare...

In [17]:
CLIENT_ID = '5PUTVTKJ5PPWLPYRGQWKFIZCN00JBDH4LROAZKP5YU3RTLD5' # your Foursquare ID
CLIENT_SECRET = 'C4OSQ3IZDJMSGQTKVVRSLIY45LU0NIBQ0FJTQGMPMJEWVRL5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5PUTVTKJ5PPWLPYRGQWKFIZCN00JBDH4LROAZKP5YU3RTLD5
CLIENT_SECRET:C4OSQ3IZDJMSGQTKVVRSLIY45LU0NIBQ0FJTQGMPMJEWVRL5


### Creating a function to get venues near each data point

In [18]:
LIMIT = 10
radius = 50

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Running the above function with my Toronto data

In [19]:
toronto_venues = getNearbyVenues(names=raw_dataset['Neighborhood'], latitudes=raw_dataset['Latitude'], longitudes=raw_dataset['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

### Looking at size and head of resulting dataframe....

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(693, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


### Looking at how many venues were returned for each neighborhood...

In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",10,10,10,10,10,10
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",10,10,10,10,10,10
...,...,...,...,...,...,...
Willowdale,16,16,16,16,16,16
Woburn,4,4,4,4,4,4
Woodbine Heights,9,9,9,9,9,9


### Each neighborhood is analyzed below...

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Quickly examining....

In [23]:
toronto_onehot.shape

(693, 179)

### Group rows by neighborhood and taking the mean of the frequency of occurrence of each category

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Willowdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Looking at new size...

In [25]:
toronto_grouped.shape

(95, 179)

### Skipping ahead, below k-means is run to cluster and visualize the data

In [ ]:
from sklearn.cluster import KMeans

# set number of clusters to 10 (arbitrary, it is out of the scope to optimize this for any reason)
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

### The above code block keeps killing my kernel, I don't understand why. 
I cannot finish the code execution because of this, but I shall include the code below regardless.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import sklearn